In [0]:
from __future__ import absolute_import
from __future__ import print_function
import sys
#reload(sys)
#sys.setdefaultencoding('utf-8')
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers.core import Dense, Dropout
from nltk.stem.wordnet import WordNetLemmatizer
import re
import itertools
import os.path
import json
from datetime import datetime

Using TensorFlow backend.


In [0]:
def k_to_one_hot(k_hot_vector):
    # This function converts k-hot target vector to one-hot target matrix
    
    classes = np.unique(k_hot_vector)
    one_hot_matrix = []
    
    for i in np.arange(len(classes)):
        row = (k_hot_vector == classes[i]).astype(int, copy = False)
        if len(one_hot_matrix) == 0:
            one_hot_matrix = row
        else:
            one_hot_matrix = np.vstack((one_hot_matrix, row))
            
    return classes, one_hot_matrix.conj().transpose()
    
    
def read_data(filename):
    # read data into lists
    
    with open(filename) as data_file:    
        data = json.load(data_file)
        
    ids, cuisines, ingredients = [], [], []
    if 'cuisine' in data[0].keys():
        for i in range(len(data)):
            ids.append(data[i]['id'])
            cuisines.append(data[i]['cuisine'])
            ingredients.append(data[i]['ingredients'])
    else:
        for i in range(len(data)):
            ids.append(data[i]['id'])
            ingredients.append(data[i]['ingredients'])    
                
    return ids, cuisines, ingredients
    
    
def create_submission(test_ids, guess):
    # create submission in proper format
    
    sub = np.transpose(np.vstack((test_ids, guess)))
    sub = np.vstack((['id', 'cuisine'], sub))
    sub_file_name = './submission_' + str(datetime.now())[0:16] +'.csv'
    sub_file_name = sub_file_name.replace(' ', '_')
    sub_file_name = sub_file_name.replace(':', '-')
    np.savetxt(sub_file_name, sub, delimiter=",", fmt="%s")
    
    return None  
    

def remove_numbers(ing):
    # remove numbers from ingredients
    
    return [[re.sub("\d+", "", x) for x in y] for y in ing]

    
def remove_special_chars(ing):
    # remove certain special characters from ingredients
   
    ing = [[x.replace("-", " ") for x in y] for y in ing] 
    ing = [[x.replace("&", " ") for x in y] for y in ing] 
    ing = [[x.replace("'", " ") for x in y] for y in ing] 
    ing = [[x.replace("''", " ") for x in y] for y in ing] 
    ing = [[x.replace("%", " ") for x in y] for y in ing] 
    ing = [[x.replace("!", " ") for x in y] for y in ing] 
    ing = [[x.replace("(", " ") for x in y] for y in ing] 
    ing = [[x.replace(")", " ") for x in y] for y in ing] 
    ing = [[x.replace("/", " ") for x in y] for y in ing] 
    ing = [[x.replace("/", " ") for x in y] for y in ing] 
    ing = [[x.replace(",", " ") for x in y] for y in ing] 
    ing = [[x.replace(".", " ") for x in y] for y in ing] 
    ing = [[x.replace(u"\u2122", " ") for x in y] for y in ing] 
    ing = [[x.replace(u"\u00AE", " ") for x in y] for y in ing] 
    ing = [[x.replace(u"\u2019", " ") for x in y] for y in ing] 

    return ing
    
    
def make_lowercase(ing):
    # make all letters lowercase for all ingredients
    
    return [[x.lower() for x in y] for y in ing]
    
    
def remove_extra_whitespace(ing):
    # removes extra whitespaces
    
    return [[re.sub( '\s+', ' ', x).strip() for x in y] for y in ing] 
    
    
def stem_words(ing):
    # word stemming for ingredients
    
    lmtzr = WordNetLemmatizer()
    
    def word_by_word(strng):
        
        return " ".join(["".join(lmtzr.lemmatize(w)) for w in strng.split()])
    
    return [[word_by_word(x) for x in y] for y in ing] 
    
    
def remove_units(ing):
    # remove certain words from ingredients
    
    remove_list = ['g', 'lb', 's', 'n']
        
    def check_word(strng):
        
        s = strng.split()
        resw  = [word for word in s if word.lower() not in remove_list]
        
        return ' '.join(resw)

    return [[check_word(x) for x in y] for y in ing] 
    

def extract_feats(ingredients, uniques):
    # each ingredient + each word as feature
    
    feats_whole = np.zeros((len(ingredients), len(uniques)))
    for i in range(len(ingredients)):
        for j in ingredients[i]:
            feats_whole[i, uniques.index(j)] = 1
            
    new_uniques = []
    for m in uniques:
        new_uniques.append(m.split())
    new_uniques = list(set(list(itertools.chain.from_iterable(new_uniques))))
    
    feats_each = np.zeros((len(ingredients), len(new_uniques))).astype(np.uint8)
    for i in range(len(ingredients)):
        for j in ingredients[i]:
            for k in j.split():
                feats_each[i, new_uniques.index(k)] = 1
            
    return np.hstack((feats_whole, feats_each)).astype(bool)
    
    
def load_model():
    # load neural net model architectiure
    
    mdl = Sequential()
    mdl.add(Dense(512, init='glorot_uniform', activation='relu', input_shape=(train_feats.shape[1],)))
    mdl.add(Dropout(0.5))
    mdl.add(Dense(128, init='glorot_uniform', activation='relu'))
    mdl.add(Dropout(0.5))
    mdl.add(Dense(20, activation='softmax'))
    mdl.compile(loss='categorical_crossentropy', optimizer='adadelta')
    
    return mdl    

In [0]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [0]:
df=pd.read_json('./train.json')
#df['ingredients_train'] = df['ingredients'].apply(','.join)
X =  df['ingredients'].values
Y = df['cuisine'].values

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2 ,random_state=2019, stratify = Y)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(31819,) (7955,) (31819,) (7955,)


In [0]:
# train_ids, train_cuisines, train_ingredients = read_data('train.json')
X_train = make_lowercase(X_train)
X_train = remove_numbers(X_train)
X_train = remove_special_chars(X_train)
X_train = remove_extra_whitespace(X_train)
X_train = remove_units(X_train)
X_train = stem_words(X_train)

# preprocess test set
# test_ids, test_cuisines, test_ingredients = read_data('test.json')
X_test = make_lowercase(X_test)
X_test = remove_numbers(X_test)
X_test = remove_special_chars(X_test)
X_test = remove_extra_whitespace(X_test)
X_test = remove_units(X_test)
X_test = stem_words(X_test)


# train_cuisines = y_train

le = LabelEncoder()
targets = le.fit_transform(y_train)
classes, targets = k_to_one_hot(targets)

# extract features

uniques = list(set([item for sublist in X_train + X_test for item in sublist]))
train_feats = extract_feats(X_train, uniques)
test_feats = extract_feats(X_test, uniques)

In [0]:
n_ensemble = 10
for ens in range(n_ensemble):
    print("\n\tTraining...", ens)
    model = load_model()

    # if model already exists, continue training
    model_name = 'model' + str(ens) + '.hdf5'
    if os.path.isfile(model_name):
        model.load_weights(model_name)

#    model.fit(train_feats, targets, nb_epoch=2500, batch_size=4096)
    model.fit(train_feats, targets, nb_epoch=2000, batch_size=4096)
    model.save_weights(model_name, overwrite=True)

# create submission out of the ensemble
preds = []
for ens in range(n_ensemble):
    print("\nSubmission", ens)
    model = load_model()

    model_name = 'model' + str(ens) + '.hdf5'
    model.load_weights(model_name)            
    preds.append(model.predict_proba(test_feats))

# final cuisine decision: argmax of sum of log probabilities  
print("\nPredicting...")      
preds = sum(np.log(preds))
guess = le.inverse_transform(np.argmax(preds, axis=1))




In [0]:
# create submission out of the ensemble
n_ensemble = 10
preds = []
for ens in range(n_ensemble):
    print("\nSubmission", ens)
    model = load_model()

    model_name = './2000ep/model' + str(ens) + '.hdf5'
    model.load_weights(model_name)            
    preds.append(model.predict_proba(test_feats))

# final cuisine decision: argmax of sum of log probabilities  
print("\nPredicting...")      
preds = sum(np.log(preds))
guess = le.inverse_transform(np.argmax(preds, axis=1))


Submission 0


/home/ncp/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:143: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(512, activation="relu", input_shape=(9452,), kernel_initializer="glorot_uniform")`
/home/ncp/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:145: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, activation="relu", kernel_initializer="glorot_uniform")`



Submission 1

Submission 2

Submission 3

Submission 4

Submission 5

Submission 6

Submission 7

Submission 8

Submission 9

Predicting...


/home/ncp/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in log
  


In [0]:
from sklearn.metrics import classification_report

print(classification_report(y_test, guess))

              precision    recall  f1-score   support

   brazilian       0.82      0.65      0.72        93
     british       0.67      0.57      0.61       161
cajun_creole       0.79      0.72      0.75       309
     chinese       0.82      0.87      0.84       535
    filipino       0.74      0.68      0.71       151
      french       0.68      0.70      0.69       529
       greek       0.82      0.76      0.79       235
      indian       0.88      0.92      0.90       601
       irish       0.69      0.54      0.61       133
     italian       0.84      0.91      0.87      1568
    jamaican       0.87      0.73      0.79       105
    japanese       0.84      0.75      0.79       284
      korean       0.84      0.78      0.81       166
     mexican       0.92      0.93      0.92      1288
    moroccan       0.84      0.84      0.84       164
     russian       0.70      0.53      0.60        98
 southern_us       0.78      0.82      0.80       864
     spanish       0.69    

In [0]:
from sklearn import metrics

score = metrics.accuracy_score(y_test, guess)
print("accuracy:   %0.3f" % score)

accuracy:   0.822
